In [11]:
from catboost import Pool
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import joblib
import warnings
import pandas as pd

warnings.filterwarnings("ignore")

In [12]:
train = pd.read_csv('../Database/train_modified4.csv', index_col='ID')
X = train.drop(columns=['대출등급'])
y = train['대출등급']

In [13]:
X

,대출금액,연간소득,부채_대비_소득_비율,최근_2년간_연체_횟수,총상환원금,총상환이자,대출기간,근로기간,주택소유상태,대출목적
ID,,,,,,,,,,
TRAIN_00000,-0.563848,-0.220218,-0.014287,-0.376102,-0.800303,-0.972784,0,9,3,1
TRAIN_00001,-0.377964,0.370332,0.087890,-0.376102,-0.436814,-0.441082,1,2,1,10
TRAIN_00002,-0.610319,0.020823,-0.321114,-0.376102,0.103276,-0.627621,0,8,1,1
TRAIN_00003,-0.377964,0.382384,-0.127783,-0.376102,-0.483274,-0.624977,0,11,1,1
TRAIN_00004,-0.029431,-0.222870,0.179044,-0.376102,-0.577932,-0.634409,1,15,3,8
...,...,...,...,...,...,...,...,...,...,...
TRAIN_96289,-0.377964,1.165767,-0.299368,-0.376102,0.147972,0.145249,0,2,1,3
TRAIN_96290,1.016166,0.382384,-0.423588,-0.376102,-0.232331,0.969667,1,2,1,10
TRAIN_96291,-0.377964,-0.099698,-0.242471,-0.376102,0.648632,-0.424781,0,0,1,3


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
cat_features=[i for i in range(6,10)]

train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
val_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)

In [15]:
cat_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth = 10,
    l2_leaf_reg=3,
    task_type = 'CPU',
    loss_function = 'MultiClass',
    boosting_type = 'Ordered'
)
print(type(cat_model))
cat_model.fit(train_pool, eval_set=val_pool, verbose=5)

joblib.dump(cat_model, '../Files/cat_model.pkl')

<class 'catboost.core.CatBoostClassifier'>
0:	learn: 1.7803480	test: 1.7818396	best: 1.7818396 (0)	total: 3.99s	remaining: 1h 6m 25s
5:	learn: 1.3313084	test: 1.3353438	best: 1.3353438 (5)	total: 19s	remaining: 52m 29s
10:	learn: 1.1163398	test: 1.1220420	best: 1.1220420 (10)	total: 31.9s	remaining: 47m 50s
15:	learn: 0.9895750	test: 0.9966672	best: 0.9966672 (15)	total: 44.9s	remaining: 45m 59s
20:	learn: 0.9059515	test: 0.9134947	best: 0.9134947 (20)	total: 57.2s	remaining: 44m 27s
25:	learn: 0.8467445	test: 0.8556044	best: 0.8556044 (25)	total: 1m 12s	remaining: 44m 59s
30:	learn: 0.8016644	test: 0.8104223	best: 0.8104223 (30)	total: 1m 26s	remaining: 45m 10s
35:	learn: 0.7625640	test: 0.7716560	best: 0.7716560 (35)	total: 1m 39s	remaining: 44m 22s
40:	learn: 0.7316355	test: 0.7415042	best: 0.7415042 (40)	total: 1m 52s	remaining: 43m 42s
45:	learn: 0.7155244	test: 0.7254696	best: 0.7254696 (45)	total: 1m 59s	remaining: 41m 24s
50:	learn: 0.6996544	test: 0.7096479	best: 0.7096479 (50

['../Files/cat_model.pkl']

In [16]:
print("CatBoost Accuracy:", accuracy_score(y_test, cat_model.predict(X_test)))

CatBoost Accuracy: 0.8596491228070176
